# Assignment-2: n-gram TF-IDF and document similarity <br>
## Tag: MDS555-2023-Assignment-2 <br>
Objective
- Learn Term Frequency
- Compute document similarity

## Your Details

In [1]:
import datetime
student_rollno = 1
student_name = 'Milan Paudel'
assignment_tag = 'MDS555-2023-Assignment-2'

In [2]:
# from checker_utils import done
def done(task):
    _date = datetime.datetime.now()
    task = task + ": " + str(_date)
    print('='*len(task), '\n', task , '\n', '='*len(task), sep='')
    pass

## Tasks <br>
### 1) Task 1:<br>
Dataset Preparation: Prepare the Nepali news dataset (hint: you can
obtain text from news websites, at least 20 different news of 2/3 different
categories). <br>
Host the dataset in the public git repository.
In your notebook data should be downloaded from git or some other public
places. <br>
No additional step should be done to get the notebook working.
You can reuse the dataset of Assignment 1 as well.

In [3]:
# TODO
import requests
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')


raw_url = f'https://github.com/1st-MadaraUchiha/NepaliNewsDataBase/blob/main/News.txt'
response = requests.get(raw_url)
if response.status_code == 200:
    file_content = response.text

    print(file_content)
else:
    print(f"Failed to retrieve the file. Status code: {response.status_code}")

nepali_strings=[file_content]
nepali_strings = ' '.join(nepali_strings)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\melan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\melan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{"payload":{"allShortcutsEnabled":false,"fileTree":{"":{"items":[{"name":"News.txt","path":"News.txt","contentType":"file"}],"totalCount":1}},"fileTreeProcessingTime":6.052401,"foldersToFetch":[],"reducedMotionEnabled":null,"repo":{"id":692507079,"defaultBranch":"main","name":"NepaliNewsDataBase","ownerLogin":"1st-MadaraUchiha","currentUserCanPush":false,"isFork":false,"isEmpty":false,"createdAt":"2023-09-16T17:32:07.000Z","ownerAvatar":"https://avatars.githubusercontent.com/u/97103435?v=4","public":true,"private":false,"isOrgOwned":false},"symbolsExpanded":false,"treeExpanded":true,"refInfo":{"name":"main","listCacheKey":"v0:1694885685.0","canEdit":false,"refType":"branch","currentOid":"b85f46ee2938576efefca7eb34b61cdd555fac3c"},"path":"News.txt","currentUser":null,"blob":{"rawLines":["न्युयोर्क, अमेरिका — संयुक्त राष्ट्रसंघ (यूएन) को ७८ औं महासभाको चहलपहल बढेको छ। हरेक वर्ष हुने बहुपक्षीय कूटनीतिको यो ‘कुम्भ मेला’ मा सहभागी हुन विभिन्न मुलुकका अधिकारी न्युयोर्कस्थित यूएनको मुख्यालयमा

In [4]:
#Tokenizing the string
tokens = word_tokenize(nepali_strings)

#using unicode to create range of regular expression
pattern = re.compile(u'[\u0900-\u097F]').pattern            #For all devenagri unicode
pattern_number = re.compile(u'[\u0966-\u096F]').pattern     #for numeric devnagari
non_devanagaric = re.compile(u'[^\u0900-\u097F]').pattern   #for all non devenagari unicode

#normalizing the tokens to remove numeric, non devanagric characters and punctuations
modified_token = []
for word in tokens:
    if re.search("।",word):
        word = re.sub("।","",word)      #replacing pudabiram "।" in a word with empty character
    if not re.search(non_devanagaric,word) and not re.search(pattern_number,word) and len(word)>0:
        modified_token.append(word)
    if re.search(non_devanagaric,word) and len(word)>0: #working on words with non devanagric characters
        word = re.sub(non_devanagaric," ",word)         #replacing non devanagric characters with space
        word = word_tokenize(word)                      #replacing the characters with space results in two or more words. Thus tokenizing to get separate words
        for sub in word:
            if not re.search(pattern_number,sub) and len(word)>0:
                modified_token.append(sub)

#filtering and removing stopwords from the token
stop_words = set(stopwords.words('nepali'))           #creating set of nepali stop words
filtered_with_stop_words= []
for token in modified_token:
    if token not in stop_words:
      filtered_with_stop_words.append(token)
filtered_with_stop_words

['न्युयोर्क',
 'अमेरिका',
 'संयुक्त',
 'राष्ट्रसंघ',
 'यूएन',
 'महासभाको',
 'चहलपहल',
 'बढेको',
 'वर्ष',
 'बहुपक्षीय',
 'कूटनीतिको',
 'कुम्भ',
 'मेला',
 'सहभागी',
 'विभिन्न',
 'मुलुकका',
 'अधिकारी',
 'न्युयोर्कस्थित',
 'यूएनको',
 'मुख्यालयमा',
 'आउन',
 'थालेका',
 'महासभामा',
 'सय',
 'सदस्य',
 'मुलुक',
 'आमन्त्रित',
 'मुलुकका',
 'राष्ट्रप्रमुख',
 'सरकार',
 'प्रमुख',
 'वा',
 'मन्त्रीस्तरीय',
 'तहमा',
 'सहभागिता',
 'हुँदै',
 'महासभाको',
 'उद्घाटन',
 'यूएनका',
 'महासचिव',
 'एन्टोनियो',
 'गुटेरेसले',
 'सेप्टेम्बर',
 'गरिसकेका',
 'कोभिड',
 'कारणले',
 'वर्षअघिको',
 'महासभा',
 'भर्चुअल',
 'त्यसपछिका',
 'दुईवटा',
 'महासभा',
 'सीमित',
 'प्रतिनिधिको',
 'सहभागितामा',
 'गरिएको',
 'महासभाका',
 'अध्यक्षता',
 'क्यारेबियन',
 'मुलुक',
 'ट्रिनिडाड',
 'एन्ड',
 'टोबागोबाट',
 'यूएनका',
 'न्युयोर्कमा',
 'स्थायी',
 'प्रतिनिधि',
 'डेनिस',
 'फ्रान्सिस',
 'नियुक्त',
 'भएका',
 'महासभाको',
 'सुरुवात',
 'सेप्टेम्बर',
 'उच्चस्तरीय',
 'सम्बोधनबाट',
 'हुनेछ',
 'महासभाको',
 'नारा',
 'विश्वासलाई',
 'पुनःस्थापित',
 'विश्

In [5]:
done('Task 1')

Task 1: 2023-09-26 01:28:16.334255


### 2) Task 2:<br>
Prepare one-gram, bi-gram, tri-gram vocabulary

In [6]:
# TODO
#creating one-gram, bi-gram and tri-gram vocabulary
one_gram = list(ngrams(filtered_with_stop_words,1))
one_gram_vocab = [" ".join(word) for word in one_gram]    #creating list of one-gram vocabulary

bi_gram = list(ngrams(filtered_with_stop_words,2))
bi_gram_vocab = [" ".join(word) for word in bi_gram]       #creating list of bi-gram vocabulary

tri_gram = list(ngrams(filtered_with_stop_words,3))
tri_gram_vocab = [" ".join(word) for word in tri_gram]     #creating list of tri-gram vocabulary
print(bi_gram_vocab)

['न्युयोर्क अमेरिका', 'अमेरिका संयुक्त', 'संयुक्त राष्ट्रसंघ', 'राष्ट्रसंघ यूएन', 'यूएन महासभाको', 'महासभाको चहलपहल', 'चहलपहल बढेको', 'बढेको वर्ष', 'वर्ष बहुपक्षीय', 'बहुपक्षीय कूटनीतिको', 'कूटनीतिको कुम्भ', 'कुम्भ मेला', 'मेला सहभागी', 'सहभागी विभिन्न', 'विभिन्न मुलुकका', 'मुलुकका अधिकारी', 'अधिकारी न्युयोर्कस्थित', 'न्युयोर्कस्थित यूएनको', 'यूएनको मुख्यालयमा', 'मुख्यालयमा आउन', 'आउन थालेका', 'थालेका महासभामा', 'महासभामा सय', 'सय सदस्य', 'सदस्य मुलुक', 'मुलुक आमन्त्रित', 'आमन्त्रित मुलुकका', 'मुलुकका राष्ट्रप्रमुख', 'राष्ट्रप्रमुख सरकार', 'सरकार प्रमुख', 'प्रमुख वा', 'वा मन्त्रीस्तरीय', 'मन्त्रीस्तरीय तहमा', 'तहमा सहभागिता', 'सहभागिता हुँदै', 'हुँदै महासभाको', 'महासभाको उद्घाटन', 'उद्घाटन यूएनका', 'यूएनका महासचिव', 'महासचिव एन्टोनियो', 'एन्टोनियो गुटेरेसले', 'गुटेरेसले सेप्टेम्बर', 'सेप्टेम्बर गरिसकेका', 'गरिसकेका कोभिड', 'कोभिड कारणले', 'कारणले वर्षअघिको', 'वर्षअघिको महासभा', 'महासभा भर्चुअल', 'भर्चुअल त्यसपछिका', 'त्यसपछिका दुईवटा', 'दुईवटा महासभा', 'महासभा सीमित', 'सीमित प्रतिनिधिक

In [7]:
done('Task 2')

Task 2: 2023-09-26 01:28:16.393733


### 3) Task 3:<br>
Compute TF-IDF vectors for each vocabulary


In [8]:
# TODO
#computing TF-IDF vectors for each one-gram, bi-gram and tri-gram vocabulary to find similarity of vocab with entire corpus
Tfidf = TfidfVectorizer()
tfid_one_gram = Tfidf.fit_transform(one_gram_vocab)
tfid_bi_gram = Tfidf.fit_transform(bi_gram_vocab)
tfid_tri_gram = Tfidf.fit_transform(tri_gram_vocab)
print(tfid_bi_gram)

  (0, 26)	1.0
  (1, 26)	1.0
  (2, 932)	1.0
  (3, 189)	0.7407980103264344
  (3, 932)	0.6717278525536932
  (4, 1122)	0.5872595351157999
  (4, 788)	0.5179692953690147
  (4, 189)	0.6219598439372923
  (5, 365)	0.7512399523445639
  (5, 1122)	0.4949988672680122
  (5, 788)	0.4365943831575509
  (6, 704)	0.5166791924675186
  (6, 365)	0.8561790771042662
  (7, 1015)	0.66986341088456
  (7, 704)	0.7424843505139371
  (8, 645)	0.5770946810365751
  (8, 720)	0.6422866761956779
  (8, 1015)	0.5044101056687922
  (9, 432)	0.6273747586231347
  (9, 645)	0.5204543616287295
  (9, 720)	0.579247934570504
  (10, 432)	1.0
  (12, 1162)	1.0
  (13, 1162)	1.0
  (14, 226)	1.0
  :	:
  (9933, 977)	1.0
  (9934, 14)	0.685121451118011
  (9934, 502)	0.7284288552892113
  (9935, 701)	0.6060715501449129
  (9935, 14)	0.45033060471637376
  (9935, 502)	0.4787965788546638
  (9935, 909)	0.44792126388216635
  (9936, 684)	0.6141397554005791
  (9936, 701)	0.6346759869598312
  (9936, 909)	0.4690615656958916
  (9937, 684)	0.70340315567760

In [9]:
done('Task 3')

Task 3: 2023-09-26 01:28:16.659513


### 4) Task 4:<br>
Compute document similarity matrix (if your document size = N , this
will result in the NxN matrix) for each vocab list.


In [10]:
# TODO
#Computing similirity matrix between different vocabulary with in corpus
one_gram_cosine = cosine_similarity(tfid_one_gram,tfid_one_gram)
bi_gram_cosine = cosine_similarity(tfid_bi_gram,tfid_bi_gram)
tri_gram_cosine = cosine_similarity(tfid_tri_gram,tfid_tri_gram)
print(one_gram_cosine)
print(bi_gram_cosine)
print(tri_gram_cosine)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[1.         1.         0.         ... 0.         0.         0.        ]
 [1.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.80023717 0.        ]
 [0.         0.         0.         ... 0.80023717 1.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[[1.         0.73740842 0.         ... 0.         0.         0.        ]
 [0.73740842 1.         0.45161307 ... 0.         0.         0.        ]
 [0.         0.45161307 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.95096367 0.7577725 ]
 [0.         0.         0.         ... 0.95096367 1.         0.79684695]
 [0.         0.         0.    

In [11]:
done('Task 4')

Task 4: 2023-09-26 01:28:17.585894


### 5) Task 5:<br>
Write your interpretation on the result of Task 4.

In task 4, I have computed document similarity matrix using method of cosine similarity for one-gram, bi-gram, tri-gram vocabulary. The symmetric matrix is created between documents and the similarity between two documents is denoted by numeric value. The value of 1 represents the two document are similar to each other in the corpus. While the value of 0 represents two document are less similar. More the value closer to 1 there is more similarity and more value is closer to 0 there is less similarity

In [12]:
done('Task 5')

Task 5: 2023-09-26 01:28:17.597431
